## COMP30024 Artificial Intelligence Part A

Author: Rin Huang, Eric Wei

In this first part of the project, you will solve a simple search-based problem on the Cachex game board.
Before you read this specification, please make sure you have carefully read the entire ‘Rules for the Game
of Cachex’ document. Although you won’t be writing an agent to play the game just yet, you should be
aiming to get very familiar with the board layout and corresponding hex coordinate system.

The aims for Project Part A are for you and your project partner to
- refresh and extend your Python programming skills
- explore some of the algorithms you have encountered in lectures, and 
- become more familiar with the Cachex task environment.

This is also a chance for you to develop fundamental Python tools for working with the game: Some of the functions and classes you create now may be helpful later (when you are building your full game-playing program for Part B of the project).

To compute this path you are asked to use an A* search and design an **admissible heuristic** to optimise its performance. There are a number of assumptions you should make:
1. The start and goal cells will always be unoccupied (but any other cells may be occupied).
2. All given cell coordinates will be within the bounds of a board of size n. More precisely, for any given cell coordinate (r: row, q: column), 0 ≤ r < n and 0 ≤ q < n. You may also assume n ≥ 1.
3. The cost of a path is defined as the number of cells that form a continuous path from the start cell to the goal cell (including these cells).
4. If there is a tie, that is, multiple minimal paths of the same cost exist on the same board configuration, any such path is a valid solution.
5. There may not always be a valid path from the 

### Import required libraries & Define global variables

In [2]:
import os
os.chdir(os.getcwd() + '/../code')

In [4]:
# import required libraries
import json
from math import pow, sqrt, fabs
from search.util import print_board
import time
from queue import PriorityQueue

In [177]:
# define global variables
RED = 'Red'
BLUE = 'Blue'
BLOCK = 'Block'

In [178]:
# obtain board data via open the sample_input.json
# this should be read via system args
with open('sample_input.json') as json_file:
    # read cachex board game data as a const variable
    BOARD_DATA = json.load(json_file)

### Define Custom Error

In [179]:
# define the error type
class Error(Exception):
    """
    Cachex AStar Path Solver Error
    """
    pass

class InvalidHeuristicError(Error):
    """
    Heuristic function must be one of the following distance formula:
    ['euclidean', 'manhatten', 'hamming']
    """
    def __init__(self):
        self.message = "Heuristic function must be one of the following distance formula: ['euclidean', 'manhatten', 'hamming']"
        super().__init__(self.message)
        
class InvalidNodeStateError(Error):
    """
    Node only have four possible state status:
    ['Red', 'Blue', 'Block', None]
    """
    
    def __init__(self):
        self.message = "Node only have four possible state status: ['Red', 'Blue', 'Block', None]"
        super().__init__(self.message)
        
class InvalidSearchPointError(Error):
    """
    Current AStar Start point or Goal point is a board obstacle which is invalid for path finding.
    """
    
    def __init__(self, board):
        self.message = "".join([
            "Current AStar Start point or Goal point is a board obstacle which is invalid for path finding.\n",
            "Start and Goal points cannot be one of the following coordinates:\n",
            f"{board.barriers}"
        ])
        super().__init__(self.message)
        
class InvalidStartError(Error):
    """
    Current start point out of board existing board dimension.
    """
    
    def __init__(self):
        self.message ="Current start point out of board existing board dimension."

        super().__init__(self.message)
        
class InvalidGoalError(Error):
    """
    Current goal point out of board existing board dimension.
    """
    
    def __init__(self):
        self.message ="Current goal point out of board existing board dimension."

        super().__init__(self.message)

### Define CachexBoard Type

In [184]:
class AStarScore:
    def __init__(self):
        self.g = float('inf')
        self.h = float('inf')
        self.f = float('inf')
        
    def update_f(self):
        """
        Update f-score based on known g-score and h-score
        """
        self.f = self.g + self.h
        
    def __repr__(self):
        return f"f(n): {self.f}, g(n): {self.g}, h(n): {self.h}"

In [191]:
class CachexBoard:
    """
    Cachex Game object functions, a CachexBoard object should initialised with a 
    dictionary which contains board data:
    
    For example:
    BOARD_DATA = {
        'n': 5, # board dimensions
        'board': [['b', 1, 0], ['b', 1, 1], ['b', 3, 2], ['b', 1, 3]], # barriers on the game board
        'start': [4, 2], # node start point
        'goal': [0, 0] # node end point
    }
    
    board = CachexBoard(data=BOARD_DATA)
    
    The final object will contain the following attributes:
    - board.n: number of dimensions
    - board.start: board start point
    - board.goal: board target end point
    - board.data: restore board data
    - board.NodeDict: dictionary object contain each node information
    - board.board: game board layout
    - board.barriers: barriers on the board
    - board.display: print the current board
    """
    def __init__(self, data):
        self.n = data['n']
        
        # check current start and goal is valid
        if data['start'][0]+1 > self.n or data['start'][1]+1 > self.n:
            raise InvalidStartError
        self.start = (data['start'][0], data['start'][1])
        
        if data['goal'][0]+1 > self.n or data['goal'][1]+1 > self.n:
            raise InvalidGoalError
            
        self.goal = (data['goal'][0], data['goal'][1])
        self.data = data
        
        self.construct_board(self.n)
        self.construct_node_dict()
        self.obtain_barrier_coord()
        
    def __repr__(self):
        return f"Cachex Board Object n: 5"
    
    def construct_board(self, n: int, inplace=True):
        """
        The function will return all valid hexagon cell coordinates in a single
        set
        input: n: int # number of the board size
        return: board: set # a set of all possilble moves
        """
        self.board = set()

        # construct cachex board
        for r in range(self.n):
            for q in range(self.n):
                self.board.add((r, q))
        if not inplace:
            return board
    
    def construct_node_dict(self):
        """
        Construct a dictionary contain
        {
            (r[0], q[0]) : HexNode(), 
            (r[1], q[1]) : HexNode(),
            ...,
            (r[n-1], q[n-1]) : HexNode()
        }
        """
        
        self.NodeDict = dict()
        
        for node in self.board:
            # check node is barrier or not
            self.NodeDict[node] = HexNode(node)
            self.NodeDict[node].find_next_moves(self.board)
    
    def obtain_barrier_coord(self):
        """
        Read board data from the json dictionary.
        if a node has a letter 'b' at the position 0, then it is a barrier
        """
        self.barriers = set()
        
        for node in self.data['board']:
            if node[0] is 'b':
                self.NodeDict[(node[1], node[2])].state = BLOCK
                self.barriers.add((node[1], node[2]))
                
    def display(self, path=None):
        """
        This function will print the current board with the given board information
        """
        # define path dictionary and board dictionary
        path_dict = dict()
        if path is not None:
            for i, p in enumerate(path):
                path_dict[p] = i + 1 
            
        board_dict = {self.start: 'Δ', self.goal: '$'}
        for barrier in self.barriers:
            board_dict[barrier] = '#'
        
        # merge path_dict and board_dict into a single dict
        # note: path_dict only contain the path info include start and goal
        # therefore update path_dict by board_dict where board_dict
        # will rewrite start and goal with their unique symbol
        board_dict = {**path_dict, **board_dict}
       
        if path:
            # define the message
            message = self.message(path)
                          
            # print game board
            print_board(n=self.n, board_dict=board_dict, message=message)
        else:
            print_board(n=self.n, board_dict=board_dict, 
                        message="\n".join(["2022 COMP30024 Artificial Intelligence Cachex Game",
                                        f"                       Group 4399 S Huang, W, Zhao"]))
        
    def message(self, path, sep='\n'):
        """
        Message generator for display function
        """
        message = ["2022 COMP30024 Artificial Intelligence Cachex Game",
                   f"                       Group 4399 S Huang, W, Zhao",
                   "--------------------------------------------------",
                   "Symbol Representation:",
                   "- Δ: AStar Search Start Point",
                   "- $: AStar Search End/Goal Point",
                   "- #: Barriers, node cannot place at here",
                   f"- [1-{len(path)}]: AStar Path Result",
                   "--------------------------------------------------",
                   f"Board Information: Start: {self.start} >>> End: {self.goal}"]
        
        # if a valid path has been found
        if path is not None:
            message.append(f"- A* Path Length: {len(path)}")
            message.append("--------------------------------------------------")
            message.append("A* Search Path:")
            
            # define the path string
            path_str = "Start -> \n"
            for p in path[1::]:
                path_str += f"{p} -> \n"
            path_str += "Goal"
            message.append(path_str)
                        
        message.append("--------------------------------------------------")
        return sep.join(message)
    
    def AStar(self, start=None, goal=None, heuristic='manhatten', p=None):
        """
        A* Path finding algorithm implementation
        if path not found, return an empty list
        """
        
        if start is None or goal is None:
            start, goal = self.start, self.goal
        
        # A* initial state validation 
        if start not in self.NodeDict or goal not in self.NodeDict or \
            start in self.barriers or goal in self.barriers:
            raise InvalidSearchPointError(self)
        
        # obtain distance calculation function from HexNode object
        distance_diff = self.NodeDict[start].distance_diff
        
        # define the required priorityQueue, g_score, f_score, h_score
        priorityQueue = PriorityQueue()
        AStarScores = {node: AStarScore() for node in board.NodeDict.keys() if board.NodeDict[node].state is None}
           
        # store explored nodes
        # explored: {node: last-position}
        # queueTracker: set(explored queue)
        explored, queueTracker, order, path = dict(), {start}, 0, []     

        # initialise the start node (f-score, insert-order, position)
        priorityQueue.put([0, order, start])

        # initialise the start state with cost 0 and 
        # distance difference based on given heuristic function
        AStarScores[start].g = 0
        AStarScores[start].h = distance_diff(point1=start, point2=goal,
                                             heuristic=heuristic, p=p)
        
        # find the path until priorityQueue is empty
        while not priorityQueue.empty(): 
            # currentNode = [f-score, insert-order, position][2]
            # pop out the first element in the queue and delete item in queueTracker
            currentNode = priorityQueue.get()[2]
            queueTracker.remove(currentNode)
            
            # if currentNode react the target goal, return path
            if currentNode == goal:
                while currentNode in explored:
                    currentNode = explored[currentNode]
                    path.append(currentNode)
                path.insert(0, goal)
                return path[::-1]
            
            
            # check state for next expanding nodes
            for nextNode in self.NodeDict[currentNode].next:
                # ignore node which node state is not empty
                if self.NodeDict[nextNode].state is None:
                    # update path if cost is lower than previous one
                    if AStarScores[currentNode].g + 1 < AStarScores[nextNode].g:
                        AStarScores[nextNode].g = AStarScores[currentNode].g + 1
                        AStarScores[nextNode].h = distance_diff(point1=nextNode, point2=goal,
                                                                heuristic=heuristic, p=p)
                        AStarScores[nextNode].update_f()
                        
                        explored[nextNode] = currentNode # update path history
                        # if no update on cost, generate a queue item and put it in priority queue
                        if nextNode not in queueTracker:
                            order += 1
                            priorityQueue.put([AStarScores[nextNode].f, order, nextNode])
                            queueTracker.add(nextNode)
                else: 
                    pass
        
        # if path is blocked, return empty list
        return []

### Define HexNode type for Hexagon Cell

In [181]:
# define the node class
class HexNode:
    """
    In Cachex game, each Hexagon cell will be represented with a object HexNode,
    where HexNode contains its coordinates, next valid moves, current hexagon cell 
    status.
    input: coords: tuple, move: list, state: string or None
    return: class HexNode
    """
    def __init__(self, coord: tuple, state=None, board=None):
        self.coord = coord
        
        # if board is known, then automatically generate next moves
        if board:
            self.next = self.find_next_moves(board=board, inplace=False)
        else:
            self.next = set()
        
        if state not in ['Red', 'Blue', 'Block', None]:
            raise InvalidNodeStateError
            
        self.state = state # state could be Red, Blue, Block or None
        
    def __repr__(self):
        return f"HexNode {self.coord} - state: {self.state}"
        
    def summary(self):
        """
        Print Node object detailed information.
        """
        print("=====================================================================")
        print(f"Current Node Position: {self.coord}")
        print(f"Current Node State: {self.state}")
        print(f"Next Possible Moves: {self.next}")
        print("=====================================================================")
        
        
    def distance_diff(self, point1: tuple, point2: tuple, heuristic='manhatten', p=None):
        """
        Calculate the distance between current node and the target hexagon cell using
        given heuristic distance function
        
        heuristic must be one of the following distance formula:
        ['euclidean', 'manhatten', 'hamming']
        """
        if heuristic not in ['euclidean', 'manhatten', 'minkowski']:
            raise InvalidHeuristicError
        
        # calculate the distance with the given heuristic distance formula
        if heuristic is 'euclidean':
            return self.minkowski(point1, point2, 2)
        elif heuristic is 'manhatten':
            return self.minkowski(point1, point2, 1)
        elif heuristic is 'minkowski':
            return self.minkowski(point1, point2, p)
    
    def minkowski(self, point1: tuple, point2: tuple, p:int):
        """
        Calculate the distance use minkowski distance formula where
        distance = (sum( abs(point1[0] - point2[0])^p, abs(point1[1] - point2[1])^p ))**(1/p)
        
        where p = 1, minkowski == manhatten distance
        where p = 2, minkowski == euclidean distance
        """
        return pow(pow(abs(point1[0] - point2[0]), p) + pow(abs(point1[1]-point2[1]), p), 1/p)
    
    def find_next_moves(self, board: CachexBoard, inplace=True, verbose=0):
        """
        Through obversation, if turn the Cachex game board from a hexagon 2D layout to a rectangle grid
        layout, a node could move in six directions except a node cannot move along the major axis.
        
        Hence for each point:
        1. check all posible moves
        2. check moves are vaild (in board)
        3. return a result list
        
        board: set # game board coordinates information to check a move is valid or not
        inplace: boolean # could return a set instead changing node attribute value
        """
        
        # define the output result
        possible_moves = set()
        
        # generate possible moves
        for r in range(self.coord[0]-1, self.coord[0]+2):
            for q in range(self.coord[1]-1,  self.coord[1]+2):
                if (r, q) in board:
                    if verbose == 1:
                        print(f"Expanding node: {self.coord}, generated next valid move {(r, q)}")
                    possible_moves.add((r, q))

        # remove the diagonal elements along the major axis
        if verbose == 1:
            print(f"Removing diagonal element {(self.coord[0]-1, self.coord[1]-1)}, {(self.coord[0]+1, self.coord[1]+1)}")
        if (self.coord[0]-1, self.coord[1]-1) in possible_moves:
            possible_moves.remove((self.coord[0]-1, self.coord[1]-1))
        if (self.coord[0]+1, self.coord[1]+1) in possible_moves:
            possible_moves.remove((self.coord[0]+1, self.coord[1]+1))
        if self.coord in possible_moves:
            possible_moves.remove(self.coord)
        
        
        # return result
        if inplace is True:
            self.next = possible_moves
            return
        
        return possible_moves

    
    def state_check(self, isState):
        """
        Return current state check result by compare the target value
        ['Red', 'Blue', 'Block', None]
        return boolean status True or False
            if is None, then the next move is valid
            if not then the next move is invalid
        """
        if isState not in ['Red', 'Blue', 'Block', None]:
            raise InvalidNodeStateError
        
        if self.state == isState:
            return True
        return False
    